In [1]:
import os
os.chdir("C:/Program Files/Stata18/utilities")
from pystata import config
config.init("mp")


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      StataNow 18.5
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2023 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-782-8272        https://www.stata.com
                                   979-696-4600        service@stata.com

Stata license: Unlimited-user 2-core network, expiring  8 Jun 2025
Serial number: 501809316934
  Licensed to: Liming Lin
               Sciences Po

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


Q1 Load STATA dataset ee2002ext.dta.

In [2]:
%%stata
cd "D:\My Drive\Sciences Po\Spring 2025\Econometrics\HW\3"
use ee2002ext.dta, clear


. cd "D:\My Drive\Sciences Po\Spring 2025\Econometrics\HW\3"
D:\My Drive\Sciences Po\Spring 2025\Econometrics\HW\3

. use ee2002ext.dta, clear

. 


Q2 Create variable log(salfr) and trim the sample below the 0.5th percentile and above the 99.5th one.
Remove observations corresponding to individuals still in school (ddipl1=7).

In [3]:
%%stata
gen lw =log(salfr+1)
_pctile lw, p(.5 99.5)
return list 
global p0050 = r(r1)
global p9950 = r(r2)
drop if lw < $p0050 | lw > $p9950 
drop if ddipl1 == 7


. gen lw =log(salfr+1)
(89,004 missing values generated)

. _pctile lw, p(.5 99.5)

. return list 

scalars:
                 r(r1) =  .6931471824645996
                 r(r2) =  10.59665966033936

. global p0050 = r(r1)

. global p9950 = r(r2)

. drop if lw < $p0050 | lw > $p9950 
(89,445 observations deleted)

. drop if ddipl1 == 7
(1,696 observations deleted)

. 


Q3 Generate age squared (age is agd).

In [4]:
%%stata
gen agd2 = agd^2

Q4 Regress log(salfr) on gender (variable s), age, age squared and ddipl1.

In [5]:
%%stata
gen male=1 if s==1
replace male=0 if s==2
tabulate ddipl1, generate(ddipl1_dum)
reg lw male agd agd2 ddipl1_dum*


. gen male=1 if s==1
(25,184 missing values generated)

. replace male=0 if s==2
(25,184 real changes made)

. tabulate ddipl1, generate(ddipl1_dum)

     ddipl1 |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |     11,650       22.05       22.05
          2 |      3,904        7.39       29.44
          3 |     16,066       30.41       59.85
          4 |      7,895       14.94       74.79
          5 |      7,177       13.58       88.37
          6 |      6,143       11.63      100.00
------------+-----------------------------------
      Total |     52,835      100.00

. reg lw male agd agd2 ddipl1_dum*
note: ddipl1_dum6 omitted because of collinearity.

      Source |       SS           df       MS      Number of obs   =    52,835
-------------+----------------------------------   F(8, 52826)     =   2045.66
       Model |  5339.99388         8  667.499234   Prob > F        =    0.0000
    Residual |  17237.1499    52,826  .3263004

Q5 Interpret the regression output.

male: Holding other variables constant, male's salary is approximately 36.77% higher than female.

agd & agd2: Holding other variables constant, one year increase in age increases the wage by (5.72-0.056*agd)%.

ddipl1_dum1: Holding other variables constant, Aucun diplôme ou CEP's salary is 82.00% less than Diplôme supérieur.

ddipl1_dum2: Holding other variables constant, BEPC seul's salary is 56.09% less than Diplôme supérieur.

ddipl1_dum3: Holding other variables constant, CAP, BEP ou autre diplôme de ce niveau's salary is 59.03 % less than Diplôme supérieur.

ddipl1_dum4: Holding other variables constant, Baccalauréat ou brevet professionnel ou autre diplôme de ce niveau's salary is 42.08% less than Diplôme supérieur.

ddipl1_dum5: Holding other variables constant, Baccalauréat + 2 ans's salary is 21.72% less than Diplôme supérieur.

_cons: Holding other variables constant, the average log of wage of female with Diplôme supérieur is 7.92.


Q6 Create global variables N, K + 1 and SSR. (Use command “ereturn”)

In [6]:
%%stata
ereturn list
global N        = e(N)
global K_plus_1 = e(df_m)+1
global SSR      = e(rss)


. ereturn list

scalars:
                  e(N) =  52835
              e(sum_w) =  52835
               e(df_m) =  8
               e(df_r) =  52826
                  e(F) =  2045.65806153759
                 e(r2) =  .2365221189753191
               e(rmse) =  .5712271818146846
                e(mss) =  5339.993875905373
                e(rss) =  17237.14985610473
               e(r2_a) =  .2364064974433424
                 e(ll) =  -45379.1928455106
               e(ll_0) =  -52508.51329657075
               e(rank) =  9

macros:
            e(cmdline) : "regress lw male agd agd2 ddipl1_dum*"
              e(title) : "Linear regression"
          e(marginsok) : "XB default"
                e(vce) : "ols"
             e(depvar) : "lw"
                e(cmd) : "regress"
         e(properties) : "b V"
            e(predict) : "regres_p"
              e(model) : "ols"
          e(estat_cmd) : "regress_estat"

matrices:
                  e(b) :  1 x 10
                  e(V) :  10 x 10
 

Q7 Use command “test” to test the null that there is no effect of education.

In [7]:
%%stata
testparm ddipl1_dum*


 ( 1)  ddipl1_dum1 = 0
 ( 2)  ddipl1_dum2 = 0
 ( 3)  ddipl1_dum3 = 0
 ( 4)  ddipl1_dum4 = 0
 ( 5)  ddipl1_dum5 = 0

       F(  5, 52826) = 2062.89
            Prob > F =    0.0000


Since the p-value is less than 0.05, we can reject the null hypothesis that education level has no effects on log of wage.

Q8 Test H0: the effects of (ddipl1==2) and (ddipl1==3) are equal. What do you conclude ?

In [8]:
%%stata
test ddipl1_dum2 = ddipl1_dum3


 ( 1)  ddipl1_dum2 - ddipl1_dum3 = 0

       F(  1, 52826) =    8.28
            Prob > F =    0.0040


Since the p-value is less than 0.05, we can reject the null hypothesis that BEPC seul and CAP, BEP ou autre diplôme de ce niveau have the same effects on log of wage.

Q9 Estimate the model under the null that the effects of (ddipl1==2) and (ddipl1==3) are equal.

In [9]:
%%stata
constraint define 1 _b[ddipl1_dum2] = _b[ddipl1_dum3]
cnsreg lw male agd agd2 ddipl1_dum*, constraint(1)


. constraint define 1 _b[ddipl1_dum2] = _b[ddipl1_dum3]

. cnsreg lw male agd agd2 ddipl1_dum*, constraint(1)
note: ddipl1_dum6 omitted because of collinearity.

Constrained linear regression                          Number of obs =  52,835
                                                       F(7, 52827)   = 2336.39
                                                       Prob > F      =  0.0000
                                                       Root MSE      =  0.5713

 ( 1)  ddipl1_dum2 - ddipl1_dum3 = 0
------------------------------------------------------------------------------
          lw | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
        male |   .3668036   .0050141    73.15   0.000     .3569759    .3766314
         agd |   .0571367   .0017824    32.06   0.000     .0536431    .0606302
        agd2 |  -.0005562    .000022   -25.29   0.000    -.0005993   -.0005131
 ddipl1_d

Q10 Regress log(salfr) on gender (variable s), age, age squared.

In [10]:
%%stata
reg lw male agd agd2


      Source |       SS           df       MS      Number of obs   =    52,837
-------------+----------------------------------   F(3, 52833)     =   1688.53
       Model |  1975.59931         3  658.533104   Prob > F        =    0.0000
    Residual |  20605.0448    52,833  .390003309   R-squared       =    0.0875
-------------+----------------------------------   Adj R-squared   =    0.0874
       Total |  22580.6441    52,836  .427372324   Root MSE        =     .6245

------------------------------------------------------------------------------
          lw | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
        male |   .3264999   .0054423    59.99   0.000     .3158329    .3371669
         agd |   .0565733   .0019433    29.11   0.000     .0527645    .0603822
        agd2 |  -.0006036    .000024   -25.17   0.000    -.0006506   -.0005566
       _cons |   7.549446    .037765   199.91   0.

Q11 Create a global variable containing SSR for that model.

In [11]:
%%stata
ereturn list
global SSR2 = e(rss)


. ereturn list

scalars:
                  e(N) =  52837
              e(sum_w) =  52837
               e(df_m) =  3
               e(df_r) =  52833
                  e(F) =  1688.532093365435
                 e(r2) =  .0874908306168676
               e(rmse) =  .62450244904602
                e(mss) =  1975.599310609145
                e(rss) =  20605.0448172369
               e(r2_a) =  .0874390158891757
                 e(ll) =  -50094.79413365701
               e(ll_0) =  -52513.59656717326
               e(rank) =  4

macros:
            e(cmdline) : "regress lw male agd agd2"
              e(title) : "Linear regression"
          e(marginsok) : "XB default"
                e(vce) : "ols"
             e(depvar) : "lw"
                e(cmd) : "regress"
         e(properties) : "b V"
            e(predict) : "regres_p"
              e(model) : "ols"
          e(estat_cmd) : "regress_estat"

matrices:
                  e(b) :  1 x 4
                  e(V) :  4 x 4
               e(

Q12 Create a global variable containing the Fisher statistic for the test that education has no effect.
Compare with Stata output.

In [12]:
%%stata
global F = (($SSR2 - $SSR) / ($K_plus_1 - 4)) / ($SSR / ($N - $K_plus_1))
display "Fisher statistic: " $F


. global F = (($SSR2 - $SSR) / ($K_plus_1 - 4)) / ($SSR / ($N - $K_plus_1))

. display "Fisher statistic: " $F
Fisher statistic: 2064.2905

. 


The resulting F-stats here is very close to the one we get in Q7, in which we directly test the joint sigificance of the 5 parameters on education dummy variables.

Q13 Ask Stata to provide the 95th percentile of the Fisher distribution with the right degrees of freedom
for the test. (Use command “display” and the Fisher CDF function.) What do you deduce?

In [13]:
%%stata
display "Critical value: " invFtail($K_plus_1 - 4, $N - $K_plus_1, 0.05)

Critical value: 2.2142687


Since the F-stats is greater than the critical value, we can reject the null hypothesis that the education level has no effects on log of wage.

Q14 Test H0: the effects of (dippl1==5) while being a woman is the same as the effect of (dippl1==3) while being a man. Follow these steps:

(a) Set up the relevant hypothesis and t-statistic

Effects of dippl1==5 while being a woman=$\beta_0+\beta_{dippl1_5}$

Effects of dippl1==3 while being a man=$\beta_0+\beta_{male}+\beta_{dippl1_3}$

Null Hypothesis: $\beta_0+\beta_{dippl1_5}-(\beta_0+\beta_{male}+\beta_{dippl1_3})=\beta_{dippl1_5}-\beta_{male}-\beta_{dippl1_3}=0$

Alternative Hypothesis: $\beta_{dippl1_5}-\beta_{male}-\beta_{dippl1_3}\ne 0$

$t-stats=\frac{\beta_5-\beta_3-\beta_{male}}{SE}$


(b) Compute its standard error using the Delta method

We define $\theta=\beta_{dippl1_5}-\beta_{male}-\beta_{dippl1_3}$

Using the Delta Method we have

 $Var(\hat{\theta})$
 
 $=\begin{bmatrix} 1 & -1 & -1 \end{bmatrix} \begin{bmatrix} Var(\beta_{dippl1_5}) & Cov(\beta_{male},\beta_{dippl1_5}) & Cov(\beta_{dippl1_3},\beta_{dippl1_5})\\Cov(\beta_{male},\beta_{dippl1_5}) & Var(\beta_{male}) & Cov(\beta_{male},\beta_{dippl1_3})\\Cov(\beta_{dippl1_3},\beta_{dippl1_5}) & Cov(\beta_{male},\beta_{dippl1_3}) & Var(\beta_{dippl1_3}) \end{bmatrix}\begin{bmatrix}1\\-1\\-1\end{bmatrix}$

And hence $SE=\sqrt{Var(\hat{\theta})}$



In [14]:
%%stata
reg lw male agd agd2 ddipl1_dum*
ereturn list
matrix A=e(V)
matrix B=e(b)
global dum5=A[8,8]
global dum3=A[6,6]
global male=A[1,1]
global male_dum5=A[1,8]
global male_dum3=A[1,6]
global dum3_dum5=A[6,8]
global b_male=B[1,1]
global b_dum3=B[1,6]
global b_dum5=B[1,8]
matrix omega =($dum5,$male_dum5,$dum3_dum5 \ $male_dum5,$male,$male_dum3 \ $dum3_dum5,$male_dum3,$dum3)
matrix J=(1,-1,-1)
matrix Delta=J*omega*J'
global V=Delta[1,1]
global t=($b_dum5-$b_dum3-$b_male)/sqrt($V)
display "t-statistic: " $t


. reg lw male agd agd2 ddipl1_dum*
note: ddipl1_dum6 omitted because of collinearity.

      Source |       SS           df       MS      Number of obs   =    52,835
-------------+----------------------------------   F(8, 52826)     =   2045.66
       Model |  5339.99388         8  667.499234   Prob > F        =    0.0000
    Residual |  17237.1499    52,826  .326300493   R-squared       =    0.2365
-------------+----------------------------------   Adj R-squared   =    0.2364
       Total |  22577.1437    52,834   .42732225   Root MSE        =    .57123

------------------------------------------------------------------------------
          lw | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
        male |   .3676793    .005023    73.20   0.000     .3578342    .3775245
         agd |   .0572084   .0017825    32.09   0.000     .0537147     .060702
        agd2 |  -.0005573    .000022   -25

(c) Conclude

Since the t-value is less than 1.96, we cannot reject the null hypothesis that the two groups have same effects on log of wage.

Found a package to solve this.

In [15]:
%%stata
lincom ddipl1_dum5-ddipl1_dum3-male


 ( 1)  - male - ddipl1_dum3 + ddipl1_dum5 = 0

------------------------------------------------------------------------------
          lw | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
         (1) |   .0054315   .0091043     0.60   0.551    -.0124131     .023276
------------------------------------------------------------------------------
